In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
from bs4 import BeautifulSoup as bs
import numpy as np

## Results All

In [19]:
all_df = pd.concat([pd.read_csv("tes1.csv"), pd.read_csv("tes2.csv"), pd.read_csv("tes3.csv"), pd.read_csv("tes4.csv")]\
              ).iloc[:,1:].drop_duplicates()

In [20]:
all_df.shape

(432, 13)

In [50]:
all_df.head()

,merchant_name,broken_link_score,link_contact_us_exist,cu_email_exist,cu_phone_number_exist,link_about_us_exist,link_tnc_exist,tnc_refund_policy_exist,contact_us_score,tnc_score,broken_links,website,label
0,CV.ASIATRIPINDONESIA,0.0,1.0,1.0,1.0,0.0,1.0,1.0,100.0,100.0,{},http://asiatrip.id/,APPROVED
1,Pt.BloomingLotusYoga,0.0,1.0,1.0,1.0,1.0,1.0,0.0,100.0,50.0,{},https://www.blooming-lotus-yoga.com,APPROVED
2,YayasanGreenSejahtera,0.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0,100.0,{},https://greensejahterafoundation.com/,APPROVED
3,PTMatairTerraSolusi,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,{},http://www.matair.co.id,APPROVED
4,SimplyMii,0.0,1.0,1.0,1.0,0.0,0.0,0.0,100.0,0.0,{},https://www.simplymii.com/,APPROVED


In [43]:
all_df.groupby("label").size()

label
APPROVED    300
REJECTED     25
REVIEWED    107
dtype: int64

In [98]:
## Inspect the negative label
# negative_df = all_df[all_df.label == 'REJECTED']
negative_df = pd.read_csv("negative_output.csv").iloc[:,1:]

In [104]:
negative_df

,merchant_name,broken_link_score,link_contact_us_exist,cu_email_exist,cu_phone_number_exist,link_about_us_exist,link_tnc_exist,tnc_refund_policy_exist,contact_us_score,tnc_score,broken_links,website,label
0,YoYoMats,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.000000,100.0,{},https://yoyomatsindonesia.myshopify.com,REJECTED
1,SisilShop,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,{'': 'None'},SisilShop.com,REJECTED
2,GammaMedia,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,{'': 'None'},http://gammamedia.co.id,REJECTED
3,ShinhwaGlobalSakuraBear,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,{'': 'None'},http://www.shinhwaglobalsakurabearinternationa...,REJECTED
4,Gitra,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,{},s.id/StartFromTheBottom,REJECTED
5,Putrimitsubishi,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,{'': 'None'},Www.mitsubishiputribali.com,REJECTED
6,TokoReza999,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,{'': 'None'},www.sdntamunti.com,REJECTED
7,wiraswasta,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,{},www.aikagames.com,REJECTED
8,BarokahLumintu,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,{'': 'None'},Www.Klink.com,REJECTED
9,MerahHitam,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,{'': 'None'},Merahhitam.com,REJECTED
